This is the code to process the output given by our C++ sorting algorithms. It also converts some of the data into a .js to be read by our React site

In [ ]:
sorted_res = {}
with open('sorted_output.txt') as topo_file:
    for line in topo_file:
        line = line.strip()
        if "City name" in line:
            city_name = line.split('City name ')[1]
            print(city_name)
            sorted_res[city_name+'_radix'] = []
            sorted_res[city_name+'_merge'] = []
            print()
        if "Merge" in line:
            sort = 'merge'
        if '# of reviews ' in line:
            review_count = line.split('# of reviews ')[1].split(' microseconds')[0]
            sorted_res[city_name+'_'+sort].append(review_count)
            print('Review_count = ' + review_count)
        if 'Prices ' in line:
            prices = line.split('Prices ')[1].split(' microseconds')[0]
            sorted_res[city_name+'_'+sort].append(prices)
            print('Prices = ' + prices)
        if 'Ratings ' in line:
            ratings = line.split('Ratings ')[1].split(' microseconds')[0]
            sorted_res[city_name+'_'+sort].append(ratings)
            print('Ratings = ' + ratings)
            print()
        if "Radix" in line:
            sort = 'radix'
        
                            

In [ ]:
#num_reviews, prices, ratings

sorted_final = {}

for key, value in sorted_res.items():
    key = key.split('_')
    name = key[0]
    sort_type = key[1]
    if (name in sorted_final):
        sorted_final[name][sort_type] = value
    else: 
        sorted_final[name] = {sort_type: value}
sorted_final

In [ ]:
import pandas as pd
import os

data = pd.read_csv('data.csv')
data['city'] = data['city'].apply(remove)

def remove(city):
    if "[" in city:
        city = city.split('[')[0]
    return city




total = 0
res = 0
ex = []

test = {}

for filename in os.listdir('cities'):
    if 'csv' in filename:
        sorts = {}
        df = pd.read_csv(f'cities/{filename}')
        city = filename.split('.csv')[0]
        price_mean = df['price_level'].mean()
        rating_mean = df['rating'].mean()
        user_ratings_total_mean = df['user_ratings_total'].mean()
        data.loc[data['city'] == city, 'price_mean'] = price_mean
        data.loc[data['city'] == city, 'rating_mean'] = rating_mean
        data.loc[data['city'] == city, 'user_ratings_total_mean'] = user_ratings_total_mean
        data.loc[data['city'] == city, 'restaurants_tracked'] = len(df)
        df1 = df[['name', 'price_level', 'rating', 'user_ratings_total', 'vicinity']]
        
        df1 = df1.fillna(0)
        city_json = df1.to_dict('records')
        list_city = list(city_json)
        test[city] = [dict(t) for t in {tuple(d.items()) for d in list_city}]
        res_tracked = len(df)
        
        data = data.sort_values(by=['price_mean'], ascending=True)
        
        

In [ ]:
f = open("all_data.js", "w")
f.write(json.dumps(test, indent = 1))
f.close()

In [ ]:
data['city'] = data['city'].apply(remove)
data.to_json('everything.json', orient='records' ,index=True)
data

In [ ]:
# https://notebook.community/captainsafia/nteract/applications/desktop/example-notebooks/pandas-to-geojson
# This code is from the above link ^^^

def df_to_geojson(df, properties, lat='lat', lon='long'):
    # create a new python dict to contain our geojson data, using geojson format
    geojson = {'type':'FeatureCollection', 'features':[]}

    # loop through each row in the dataframe and convert each row to geojson format
    for _, row in df.iterrows():
        # create a feature template to fill in
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}

        # fill in the coordinates
        feature['geometry']['coordinates'] = [row[lon],row[lat]]

        # for each column, get the value and add it as a new feature property
        for prop in properties:
            feature['properties'][prop] = row[prop]
        
        
        # add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson['features'].append(feature)
    
    return geojson

import json

res =json.dumps( df_to_geojson(data, ['city', 'state', 'price_mean', 'rating_mean', 'user_ratings_total_mean', 'restaurants_tracked']), indent=2)

with open('map.geojson', 'w') as f:
    f.write(res)